In [24]:
import sys
import os
import json
from groq import Groq
from dotenv import load_dotenv
import time

sys.path.append(os.path.abspath("..")) 

load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
client = Groq(api_key=GROQ_API_KEY)

from src.llm_engine import analyse_image
from src.ocr_engine import process_with_easyocr, parse_ocr_with_llm
from src.utils import clean_json_output

In [25]:
# MODE OCR + LLM 
def run_ocr_test(image_path, schema=None):
    start_time = time.time()
    with open(image_path, "rb") as f:
        img_bytes = f.read()
    
    # OCR brut
    raw_text, _ = process_with_easyocr(img_bytes)
    
    # parsing LLM
    json_str = parse_ocr_with_llm(raw_text, client_groq=client, schema_json=schema)
    
    # clean and parse JSON
    end_time = time.time()
    print(f"OCR + LLM processing time: {end_time - start_time:.2f} seconds")
    return json.loads(clean_json_output(json_str))

In [26]:
image_path ="/Users/tiago/Documents/esilvA5/ecole/Project-LLM/IDP_GenAI_Project/examples/phpCDwGn0.jpg"
target_schema = "/Users/tiago/Documents/esilvA5/ecole/Project-LLM/IDP_GenAI_Project/schemas/id_card_schema.json"
with open(target_schema) as f:
    schema = json.dumps(json.load(f))

In [27]:
run_ocr_test(image_path, schema=None)

OCR + LLM processing time: 6.36 seconds


{'nationalite': 'FRA',
 'carteNationalite': 'IDENTITY CARD',
 'nom': 'CHEVALLIER',
 'prenom': 'Gisèle Audrey',
 'sexe': 'F',
 'dateDeNaissance': '01 04 1995',
 'lieuDeNaissance': 'BORDEAUX',
 'nomDUsage': 'vve DUBOIS',
 'numDocument': 'T7X62Tz79',
 'dateExpir': '27 01 2031'}